In [187]:
# Import all libraries
import pandas as pd


# Import data from xls
og_df = pd.read_excel('GSAF5.xls')

# Check for basic shape and some values
print(og_df.shape)
print(og_df.head(5))

#Let's create a copy to work with and rename columns
df = og_df.copy()
df.columns = df.columns.str.replace(' ', '').str.lower()
print(df.columns)

(6969, 23)
          Date    Year        Type    Country              State  \
0  15 Mar 2024  2024.0  Unprovoked  AUSTRALIA         Queensland   
1  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
2  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
3  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
4  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   

                           Location  Activity                 Name Sex  Age  \
0                     Bargara Beach  Swimming       Brooklyn Sauer   F   13   
1                Old Man's, Waikiki   Surfing        Matthew White   M  NaN   
2                    Rainbows, Oahu  Swimming                  NaN   F   11   
3        Sandlnd Island, Jurian Bay       NaN               female   F   46   
4  Vaitarna River, Palghar District   Fishing  Vicky Suresh Govari   M   32   

   ...        Species                      Source  pdf href formula href  \
0  ...     Tiger shark      Y

In [188]:
#this columns don't seem to hold relevant information for this project
df.drop(columns=['unnamed:21'], inplace=True)
df.drop(columns=['unnamed:22'], inplace=True)
df.drop(columns=['unnamed:11'], inplace=True)


In [189]:
#Let's drop duplicates and missing values
print(df.isnull().sum())

print(df.drop_duplicates(inplace=True))

(df.dropna(subset=["date"],axis=0,inplace=True))


date               25
year               27
type               43
country            75
state             507
location          590
activity          611
name              245
sex               604
age              3019
injury             60
time             3551
species          3157
source             44
pdf               170
hrefformula       150
href              173
casenumber        171
casenumber.1      172
originalorder     170
dtype: int64
None


In [190]:
print("Before\n", df.eq(" ").sum())
df = df[df!= " "] #Very low number of spaces so we can drop this outliers
print("After\n", df.eq(" ").sum())


Before
 date             0
year             0
type             0
country          0
state            0
location         0
activity         1
name             0
sex              0
age              2
injury           1
time             2
species          7
source           0
pdf              0
hrefformula      0
href             0
casenumber       0
casenumber.1     0
originalorder    0
dtype: int64
After
 date             0
year             0
type             0
country          0
state            0
location         0
activity         0
name             0
sex              0
age              0
injury           0
time             0
species          0
source           0
pdf              0
hrefformula      0
href             0
casenumber       0
casenumber.1     0
originalorder    0
dtype: int64


In [192]:
#Country , State , Location - We really only need one of these to identify the place, so lets drop all that have all 3 missing
print(df.shape)
df = df[df!= " "]
df = df[(df["state"].isna()==False) | (df["country"].isna()==False) | (df["location"].isna()==False)] #Removes all True|True|True which have na/na/na as the location
print(df.shape)
print(df["country"].value_counts())


(6944, 20)
(6915, 20)
country
USA                   2538
AUSTRALIA             1481
SOUTH AFRICA           597
NEW ZEALAND            144
BAHAMAS                136
                      ... 
PUERTO RICO              1
RED SEA                  1
Coral Sea                1
BRITISH ISLES            1
CEYLON (SRI LANKA)       1
Name: count, Length: 225, dtype: int64


In [193]:

top_five = df["country"].value_counts().head(5)
print(top_five)
print(df.shape)


country
USA             2538
AUSTRALIA       1481
SOUTH AFRICA     597
NEW ZEALAND      144
BAHAMAS          136
Name: count, dtype: int64
(6915, 20)


In [194]:
mini_df = df[df["country"] == "USA"]

#print(mini_df.State.value_counts())

us_states_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

#state_series = mini_df["State"].value_counts(dropna=False)

mini_df["state"] = mini_df["state"].str.strip()

mini_df["state"].replace(us_states_codes, inplace=True)

#disregard states with less that 10 occurences (outliers)
state_counts = mini_df["state"].value_counts(dropna=False)

states_to_keep = state_counts[state_counts > 10].index
# Filter the DataFrame
USA_df = mini_df[mini_df["state"].isin(states_to_keep)]

print(USA_df["state"].value_counts(dropna=False))
print(USA_df["state"].value_counts().sum())



state
FL             1179
HI              341
CA              323
SC              172
NC              121
TX               78
NJ               57
NY               48
OR               31
VA               19
LA               18
MA               18
GA               17
AL               17
Puerto Rico      17
Name: count, dtype: int64
2456


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15984\709207919.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_df["state"] = mini_df["state"].str.strip()
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15984\709207919.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mini_df["sta

In [195]:
#Florida Dataframe

FL_df = mini_df[mini_df["state"] == "FL"]
print(FL_df.shape)

print(FL_df.type.value_counts())
#print(FL_df.year.value_counts())

years = FL_df.year.value_counts()

for i in range(2000,2024):
    print(f"{i} -> {years[i]}")
    
FL_df.head(0)


(1179, 20)
type
Unprovoked             982
Provoked               101
Invalid                 68
Watercraft              13
Sea Disaster             6
Questionable             4
Unconfirmed              1
Under investigation      1
Name: count, dtype: int64
2000 -> 35
2001 -> 38
2002 -> 31
2003 -> 45
2004 -> 12
2005 -> 24
2006 -> 28
2007 -> 39
2008 -> 37
2009 -> 20
2010 -> 18
2011 -> 16
2012 -> 32
2013 -> 27
2014 -> 33
2015 -> 33
2016 -> 34
2017 -> 34
2018 -> 17
2019 -> 26
2020 -> 23
2021 -> 30
2022 -> 24
2023 -> 23


,date,year,type,country,state,location,activity,name,sex,age,injury,time,species,source,pdf,hrefformula,href,casenumber,casenumber.1,originalorder


In [196]:

us = USA_df.year.value_counts()
florida = FL_df.year.value_counts()

i = 2000

for i in range(2000,2024):
    print(florida[i]/us[i]*100)

# this refers to the percent of shark attacks in the USA that happened in florida from 2000 onwards

# USA_df - already only has the USA values in it
# FL_df - already only has the FL values in it
# df - is the main dataframe with all values


68.62745098039215
65.51724137931035
64.58333333333334
80.35714285714286
34.285714285714285
47.05882352941176
53.84615384615385
60.0
62.71186440677966
48.78048780487805
47.368421052631575
33.33333333333333
50.0
45.0
53.2258064516129
45.20547945205479
52.307692307692314
52.307692307692314
40.476190476190474
46.42857142857143
53.48837209302325
58.82352941176471
48.97959183673469
51.11111111111111


In [197]:
FL_df.location.value_counts()
# clear major location

location
New Smyrna Beach, Volusia County                             191
Daytona Beach, Volusia County                                 31
Ponce Inlet, Volusia County                                   28
Melbourne Beach, Brevard County                               20
Cocoa Beach, Brevard  County                                  18
                                                            ... 
Quarter mile south of Ponce de Leon Inlet, Volusia County      1
Off Zelda Boulevard, Daytona Beach, Volusia Countyy            1
Tigertail Beach, Collier County                                1
Marco Island, Collier County                                   1
Mosquito Inlet (Ponce Inlet), Volusia County                   1
Name: count, Length: 551, dtype: int64

In [198]:
fatal = FL_df[FL_df["injury"].str.lower().str.contains('fatal', na=False)]
non_fatal = FL_df[~FL_df["injury"].str.lower().str.contains('fatal', na=False)]

#print(fatal.injury.value_counts()) 
print(fatal.shape) #53
#print(non_fatal.injury.value_counts())
print(non_fatal.shape) #1126

#print(FL_df.type.value_counts())
#print(FL_df.species.value_counts())
#print(FL_df.location.value_counts())
#print(FL_df.activity.value_counts())

print(FL_df.activity.value_counts())


(53, 20)
(1126, 20)
activity
Surfing                         415
Swimming                        151
Wading                           76
Fishing                          46
Standing                         41
                               ... 
Standing alongside surfboard      1
Standing / surfing                1
Swimming / Body surfing           1
Surfing / Wading                  1
Canoeing                          1
Name: count, Length: 213, dtype: int64


In [199]:
df_final_activities_US = FL_df
df_final_activities_US["activity"] = df_final_activities_US["activity"].fillna("Unknown")

print(df_final_activities_US["activity"].isna().sum())

df_final_activities_US["activity"] = df_final_activities_US["activity"].str.replace("," , "")
df_final_activities_US["activity"] = df_final_activities_US["activity"].str.replace("?" , "")



0


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15984\964545945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_activities_US["activity"] = df_final_activities_US["activity"].fillna("Unknown")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15984\964545945.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_activities_US["activity"] = df_final_activities_US["activity"].str.replace("," , "")
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15984\964545945.py:7: SettingWithCopyWarning: 
A value is

In [200]:
def replace_activity(activity):
    # Convert the activity to lowercase and split into words
    words = activity.lower().split()
    # Check for the exact words and replace the activity's value by a broader category
    if "fishing" in words:
        return "Fishing"
    elif "spearfishing" in words:
        return "Fishing"
    elif "fisherman" in words:
        return "Fishing"
    elif "surfing" in words:
        return "Surfing"
    elif "swimming" in words:
        return "Swimming"
    elif "wading" in words:
        return "Wading"
    elif "floating" in words:
        return "Floating"
    elif "diving" in words:
        return "Diving"
    return activity

In [201]:
# Apply the function to the Series
df_final_activities_US["activity"] = df_final_activities_US["activity"].apply(replace_activity)

df_final_activities_US["activity"].value_counts()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15984\1469196261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_activities_US["activity"] = df_final_activities_US["activity"].apply(replace_activity)


activity
Surfing                        455
Swimming                       174
Fishing                        117
Wading                          83
Unknown                         75
                              ... 
Crouching in 2' of water         1
Playing soccer in the water      1
Shrimping                        1
Playing on a sandbar             1
Canoeing                         1
Name: count, Length: 109, dtype: int64

In [203]:
FL_df["activity"].value_counts()

activity
Surfing                        455
Swimming                       174
Fishing                        117
Wading                          83
Unknown                         75
                              ... 
Crouching in 2' of water         1
Playing soccer in the water      1
Shrimping                        1
Playing on a sandbar             1
Canoeing                         1
Name: count, Length: 109, dtype: int64